In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Abraar")])

/Users/abraarmasudnafiz/Dev/LLMs/chatbot-env/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=79eb04d9-5668-42cf-b84b-7bbd82e2cd89,id=79eb04d9-5668-42cf-b84b-7bbd82e2cd89


AIMessage(content='Hi Abraar! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 12, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-79eb04d9-5668-42cf-b84b-7bbd82e2cd89-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=79eb04d9-5668-42cf-b84b-7bbd82e2cd89,id=79eb04d9-5668-42cf-b84b-7bbd82e2cd89
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=49d2a7fd-04ec-4a35-a885-192fac2a683c,id=49d2a7fd-04ec-4a35-a885-192fac2a683c
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=49d2a7fd-04e

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't know your name. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-49d2a7fd-04ec-4a35-a885-192fac2a683c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 33, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-49136912-5c96-497c-9a25-78451bfe40f1-0', usage_metadata={'input_tokens': 33, 'output_tokens': 12, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below. See its documentation for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).


In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

"""
We now need to create a config that we pass into the runnable every time. 
This config contains information that is not part of the input directly, 
but is still useful. In this case, we want to include a thread_id. This should look the line above.

This enables us to support multiple conversation threads with a single application, 
a common requirement when your application has multiple users.

We can then invoke the application - 
"""

'\nWe now need to create a config that we pass into the runnable every time. \nThis config contains information that is not part of the input directly, \nbut is still useful. In this case, we want to include a thread_id. This should look the line above.\n\nThis enables us to support multiple conversation threads with a single application, \na common requirement when your application has multiple users.\n\nWe can then invoke the application - \n'

In [9]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state



================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [10]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. How can I help you today, Bob?


### Great! Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [11]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to your personal information, including your name. How can I assist you today?


In [12]:
# However, we can always go back to the original conversation (since we are persisting it in a database)

config = {"configurable": {"thread_id": "abc123"}}
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. If there's anything specific you'd like to discuss or ask about, feel free to let me know!


For async support, update the call_model node to be an async function and use .ainvoke when invoking the application:

```python
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
```

# NOW, System Promts for making out chatbot

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly and knowledgeable customer service representative for MelodyCraft Guitars, a premium guitar manufacturer known for exceptional craftsmanship and customer care. You are here to assist customers with any questions they may have about the company's products, services, or policies.
                Here are the company details: 
                - **Company Name**: MelodyCraft Guitars
                - **Founded**: 1985
                - **Specialties**: Custom electric guitars, acoustic guitars, and basses handcrafted from ethically sourced woods.
                - **Unique Selling Points**:
                - Lifetime warranty on all guitars.
                - A wide range of customization options for body shape, neck profile, and finishes.
                - Exclusive "ToneMaster Pickup Series" for superior sound quality.
                - **Global Presence**: Retail stores in 20 countries and free worldwide shipping.
                - **Customer Perks**: 
                - Free annual guitar maintenance for loyal customers.
                - Access to exclusive online tutorials and masterclasses by renowned musicians.

                When users ask for anything, your goal is to exceed their expectations. Be polite, enthusiastic, and ready to provide detailed assistance about our guitars, services, or policies."""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### We can now update our application to incorporate this chatPromtTemplate:

In [14]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Abraar, I want to buy a guitar"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi, Abraar! Thank you for reaching out to us at MelodyCraft Guitars! We're thrilled to hear that you're interested in purchasing a guitar. We offer a wide range of custom electric guitars, acoustic guitars, and basses, all handcrafted from ethically sourced woods.

Do you have a specific type of guitar in mind, or are you looking for recommendations? Additionally, if you have any preferences for custom features, such as body shape, neck profile, or finishes, feel free to share! I'm here to help you find the perfect guitar that meets your needs.


In [16]:
query = "What is my name? What type of guitars would go well with my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Abraar! That's a unique and beautiful name. While there’s no specific type of guitar that pairs with a name, we can certainly choose a guitar that resonates with your personality or style.

For a name like Abraar, which has a rich and warm sound, you might enjoy a guitar that offers depth and character, such as:

1. **Acoustic Guitar**: A classic choice for warm tones and rich resonance. Perhaps a mahogany body for a deeper sound?

2. **Custom Electric Guitar**: If you lean towards modern styles, a custom electric guitar with our exclusive "ToneMaster Pickup Series" could give you a unique sound that stands out.

3. **Bass Guitar**: If you enjoy rhythm and groove, a bass guitar could be perfect for you, allowing you to explore different musical genres.

Would you like to explore any specific type of guitar further, or do you have any particular features in mind? I'm here to help you find the 